<a href="https://colab.research.google.com/github/Ayooluwa29/LaunchPad_Python_Task/blob/main/Airbnb_Listings_Exploratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Airbnb San Francisco Listing Analysis**

In [1]:
# import libraries
import pandas as pd
import seaborn as sb
import os
import requests
import logging as lgn
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# **Extracting  San Fransisco  Airbnb listing records in preparation for transformation and analysis**

In [2]:
url = "https://data.insideairbnb.com/united-states/ca/san-francisco/2025-09-01/data/listings.csv.gz"
print(f"\nLoading data from: {url}")


Loading data from: https://data.insideairbnb.com/united-states/ca/san-francisco/2025-09-01/data/listings.csv.gz


In [3]:
print(f"\nLoading data from: {url}")

try:
  df = pd.read_csv(url, compression='gzip', low_memory=False)
  print(f" Shape: {df.shape[0]} rows x {df.shape[1]} columns")
except Exception as e:
  print(f"x Error loadind data: {e}")
  exit()


Loading data from: https://data.insideairbnb.com/united-states/ca/san-francisco/2025-09-01/data/listings.csv.gz
 Shape: 7780 rows x 79 columns


# **Data Exploration:**
##Exploring the records of the listings and showing basic information

In [4]:
print("\n--- DataFrame Info ---")
print(df.info())


--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7780 entries, 0 to 7779
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            7780 non-null   int64  
 1   listing_url                                   7780 non-null   object 
 2   scrape_id                                     7780 non-null   int64  
 3   last_scraped                                  7780 non-null   object 
 4   source                                        7780 non-null   object 
 5   name                                          7780 non-null   object 
 6   description                                   7688 non-null   object 
 7   neighborhood_overview                         4683 non-null   object 
 8   picture_url                                   7780 non-null   object 
 9   host_id                                

In [5]:
print("--- First 5 Rows ---")
print(df.head())

--- First 5 Rows ---
     id                        listing_url       scrape_id last_scraped  \
0   958   https://www.airbnb.com/rooms/958  20250901181253   2025-09-01   
1  5858  https://www.airbnb.com/rooms/5858  20250901181253   2025-09-01   
2  8014  https://www.airbnb.com/rooms/8014  20250901181253   2025-09-01   
3  8142  https://www.airbnb.com/rooms/8142  20250901181253   2025-09-01   
4  8339  https://www.airbnb.com/rooms/8339  20250901181253   2025-09-01   

        source                                               name  \
0  city scrape              Bright, Modern Garden Unit - 1BR/1BTH   
1  city scrape                                 Creative Sanctuary   
2  city scrape     female HOST quiet fast internet market parking   
3  city scrape  *FriendlyRoom Apt. Style -UCSF/USF - San Franc...   
4  city scrape                    Historic Alamo Square Victorian   

                                         description  \
0  Our bright garden unit overlooks a lovely back...   
1

In [6]:
print("--- Statistical Summary ---")
print(df.describe())

--- Statistical Summary ---
                 id     scrape_id       host_id  host_listings_count  \
count  7.780000e+03  7.780000e+03  7.780000e+03          7779.000000   
mean   5.423931e+17  2.025090e+13  1.949055e+08           293.905901   
std    5.626811e+17  2.199360e+00  2.162753e+08          1014.788247   
min    9.580000e+02  2.025090e+13  1.169000e+03             1.000000   
25%    2.826221e+07  2.025090e+13  1.041522e+07             1.000000   
50%    5.743371e+17  2.025090e+13  9.089162e+07             4.000000   
75%    1.074843e+18  2.025090e+13  3.907730e+08            31.000000   
max    1.499678e+18  2.025090e+13  7.160698e+08          5445.000000   

       host_total_listings_count  neighbourhood_group_cleansed     latitude  \
count                7779.000000                           0.0  7780.000000   
mean                  389.589279                           NaN    37.770925   
std                  1335.978069                           NaN     0.023160   
min    

In [7]:
print("\n--- Missing Values Summary ---")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing[missing > 0],
    'Missing_Percentage': missing_pct[missing > 0]
}).sort_values('Missing_Count', ascending=False)
print(missing_df)


--- Missing Values Summary ---
                              Missing_Count  Missing_Percentage
neighbourhood_group_cleansed           7780          100.000000
calendar_updated                       7780          100.000000
neighborhood_overview                  3097           39.807198
neighbourhood                          3097           39.807198
host_about                             3084           39.640103
license                                2647           34.023136
price                                  1985           25.514139
estimated_revenue_l365d                1985           25.514139
beds                                   1958           25.167095
bathrooms                              1934           24.858612
review_scores_checkin                  1871           24.048843
review_scores_location                 1871           24.048843
review_scores_value                    1871           24.048843
reviews_per_month                      1870           24.035990
first_re

# **DATA CLEANING**:
##Cleaning the DataFrame by converting data types, handling missing values,and removing irrelevant rows.

In [8]:
# Create a copy for cleaning
df_clean = df.copy()
initial_rows = len(df_clean)
print(f"\nInitial row count: {initial_rows}")

# Convert Price Fields to Float
print("\n--- Cleaning Price Fields ---")
price_columns = ['price']

# Check if these columns exist in the dataset
price_columns = [col for col in price_columns if col in df_clean.columns]

for col in price_columns:
    if col in df_clean.columns:
        print(f"Converting {col}...")
        # Remove $ and commas, convert to float
        df_clean[col] = df_clean[col].astype(str).str.replace('$', '', regex=False)
        df_clean[col] = df_clean[col].str.replace(',', '', regex=False)
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        print(f"  ✓ {col} converted to float")


# Parse Date Columns
print("\n--- Parsing Date Columns ---")
date_columns = ['last_review']
date_columns = [col for col in date_columns if col in df_clean.columns]

for col in date_columns:
    if col in df_clean.columns:
        print(f"Parsing {col}...")
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        print(f"  ✓ {col} converted to datetime")


Initial row count: 7780

--- Cleaning Price Fields ---
Converting price...
  ✓ price converted to float

--- Parsing Date Columns ---
Parsing last_review...
  ✓ last_review converted to datetime


In [9]:
# Handle Missing Values
print("\n--- Handling Missing Values ---")

# For reviews_per_month: Fill with 0 (indicates no reviews)
if 'reviews_per_month' in df_clean.columns:
    before_na = df_clean['reviews_per_month'].isna().sum()
    df_clean['reviews_per_month'] = df_clean['reviews_per_month'].fillna(0)
    print(f"  ✓ reviews_per_month: Filled {before_na} missing values with 0")

# For host_name: Fill with 'Unknown'
if 'host_name' in df_clean.columns:
    before_na = df_clean['host_name'].isna().sum()
    df_clean['host_name'] = df_clean['host_name'].fillna('Unknown')
    print(f"  ✓ host_name: Filled {before_na} missing values with 'Unknown'")

# For neighbourhood_group: Fill with 'Unknown'
if 'neighbourhood_group' in df_clean.columns:
    before_na = df_clean['neighbourhood_group'].isna().sum()
    df_clean['neighbourhood_group'] = df_clean['neighbourhood_group'].fillna('Unknown')
    print(f"  ✓ neighbourhood_group: Filled {before_na} missing values with 'Unknown'")



--- Handling Missing Values ---
  ✓ reviews_per_month: Filled 1870 missing values with 0
  ✓ host_name: Filled 1 missing values with 'Unknown'


In [10]:
# Removing Irrelevant Rows
print("\n--- Removing Irrelevant Rows ---")

# Remove listings with zero or missing availability
if 'availability_365' in df_clean.columns:
    before = len(df_clean)
    df_clean = df_clean[df_clean['availability_365'] > 0]
    removed = before - len(df_clean)
    print(f"  ✓ Removed {removed} rows with zero availability")

# Remove listings with zero or missing price
if 'price' in df_clean.columns:
    before = len(df_clean)
    df_clean = df_clean[df_clean['price'] > 0]
    removed = before - len(df_clean)
    print(f"  ✓ Removed {removed} rows with zero or missing price")

final_rows = len(df_clean)
print(f"\nFinal row count after cleaning: {final_rows}")
print(f"Total rows removed: {initial_rows - final_rows} ({((initial_rows - final_rows) / initial_rows * 100):.2f}%)")


--- Removing Irrelevant Rows ---
  ✓ Removed 1175 rows with zero availability
  ✓ Removed 847 rows with zero or missing price

Final row count after cleaning: 5758
Total rows removed: 2022 (25.99%)


# DATA ENRICHMENT

In [11]:
# Create price_per_booking column
print("\n--- Creating price_per_booking Column ---")
if 'price' in df_clean.columns and 'minimum_nights' in df_clean.columns:
    df_clean['price_per_booking'] = df_clean['price'] * df_clean['minimum_nights']
    print(f"  ✓ price_per_booking created")
    print(f"    Mean price per booking: ${df_clean['price_per_booking'].mean():.2f}")
    print(f"    Median price per booking: ${df_clean['price_per_booking'].median():.2f}")
else:
    print("  ✗ Required columns not found for price_per_booking")

# Create availability categories
print("\n--- Creating Availability Categories ---")
if 'availability_365' in df_clean.columns:
    def categorize_availability(days):
        if days > 300:
            return 'Full-time'
        elif days >= 100:
            return 'Part-time'
        else:
            return 'Rare'

    df_clean['availability_category'] = df_clean['availability_365'].apply(categorize_availability)
    print("  ✓ availability_category created")
    print("\n  Distribution:")
    print(df_clean['availability_category'].value_counts())
    print("\n  Percentage Distribution:")
    print(df_clean['availability_category'].value_counts(normalize=True) * 100)
else:
    print("  ✗ availability_365 column not found")


--- Creating price_per_booking Column ---
  ✓ price_per_booking created
    Mean price per booking: $3886.96
    Median price per booking: $1173.50

--- Creating Availability Categories ---
  ✓ availability_category created

  Distribution:
availability_category
Part-time    2757
Full-time    1919
Rare         1082
Name: count, dtype: int64

  Percentage Distribution:
availability_category
Part-time    47.881209
Full-time    33.327544
Rare         18.791247
Name: proportion, dtype: float64


### **SUMMARY**

In [12]:
print("\n--- Cleaned DataFrame Info ---")
print(df_clean.info())


--- Cleaned DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 5758 entries, 0 to 7779
Data columns (total 81 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            5758 non-null   int64         
 1   listing_url                                   5758 non-null   object        
 2   scrape_id                                     5758 non-null   int64         
 3   last_scraped                                  5758 non-null   object        
 4   source                                        5758 non-null   object        
 5   name                                          5758 non-null   object        
 6   description                                   5693 non-null   object        
 7   neighborhood_overview                         3509 non-null   object        
 8   picture_url                              

In [13]:
print("\n--- Cleaned Data Summary Statistics ---")
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
print(df_clean[numeric_cols].describe())


--- Cleaned Data Summary Statistics ---
                 id     scrape_id       host_id  host_listings_count  \
count  5.758000e+03  5.758000e+03  5.758000e+03          5757.000000   
mean   5.472225e+17  2.025090e+13  1.903056e+08           301.350530   
std    5.677932e+17  1.214949e+00  2.130328e+08          1068.304231   
min    9.580000e+02  2.025090e+13  1.169000e+03             1.000000   
25%    2.753648e+07  2.025090e+13  1.037419e+07             1.000000   
50%    5.750729e+17  2.025090e+13  8.986820e+07             3.000000   
75%    1.078455e+18  2.025090e+13  3.896978e+08            23.000000   
max    1.499678e+18  2.025090e+13  7.160698e+08          5445.000000   

       host_total_listings_count  neighbourhood_group_cleansed     latitude  \
count                5757.000000                           0.0  5758.000000   
mean                  362.423484                           NaN    37.769182   
std                  1271.770271                           NaN     0.0237

In [14]:
print("\n--- First 10 Rows of Cleaned Data ---")
print(df_clean.head(10))


--- First 10 Rows of Cleaned Data ---
       id                         listing_url       scrape_id last_scraped  \
0     958    https://www.airbnb.com/rooms/958  20250901181253   2025-09-01   
1    5858   https://www.airbnb.com/rooms/5858  20250901181253   2025-09-01   
2    8014   https://www.airbnb.com/rooms/8014  20250901181253   2025-09-01   
3    8142   https://www.airbnb.com/rooms/8142  20250901181253   2025-09-01   
4    8339   https://www.airbnb.com/rooms/8339  20250901181253   2025-09-01   
5   10537  https://www.airbnb.com/rooms/10537  20250901181253   2025-09-01   
6   10578  https://www.airbnb.com/rooms/10578  20250901181253   2025-09-01   
7   11490  https://www.airbnb.com/rooms/11490  20250901181253   2025-09-01   
9   12042  https://www.airbnb.com/rooms/12042  20250901181253   2025-09-01   
10  12522  https://www.airbnb.com/rooms/12522  20250901181253   2025-09-01   

         source                                               name  \
0   city scrape              Bri

In [15]:
# Key insights
print("\n--- Key Insights ---")
if 'price' in df_clean.columns:
    print(f"Average nightly price: ${df_clean['price'].mean():.2f}")
    print(f"Median nightly price: ${df_clean['price'].median():.2f}")
    print(f"Price range: ${df_clean['price'].min():.2f} - ${df_clean['price'].max():.2f}")


--- Key Insights ---
Average nightly price: $378.80
Median nightly price: $170.00
Price range: $29.00 - $50000.00


In [16]:
if 'room_type' in df_clean.columns:
    print(f"\nRoom type distribution:")
    print(df_clean['room_type'].value_counts())


Room type distribution:
room_type
Entire home/apt    3736
Private room       1919
Hotel room           76
Shared room          27
Name: count, dtype: int64


In [17]:
if 'neighbourhood_group' in df_clean.columns and df_clean['neighbourhood_group'].nunique() > 1:
    print(f"\nNeighbourhood group distribution:")
    print(df_clean['neighbourhood_group'].value_counts().head(10))

In [18]:
print("\n" + "=" * 80)
print("DATA PROCESSING COMPLETE!")
print("=" * 80)
print(f"\nFinal dataset: {df_clean.shape[0]} rows × {df_clean.shape[1]} columns")
print("\nCleaned dataframe is available as: df_clean")


DATA PROCESSING COMPLETE!

Final dataset: 5758 rows × 81 columns

Cleaned dataframe is available as: df_clean


## **ANALYSIS**

In [19]:
# Question 1: Top 10 most expensive neighborhoods by average price
print("Q1: What are the top 10 most expensive neighborhoods by average price?")
print("=" * 80)
if 'neighbourhood' in df_clean.columns and 'price' in df_clean.columns:
    top_10_neighborhoods = df_clean.groupby('neighbourhood')['price'].agg(['mean', 'count']).sort_values('mean', ascending=False).head(10)
    top_10_neighborhoods.columns = ['Average_Price', 'Listing_Count']
    print("\nTop 10 Most Expensive Neighborhoods:")
    print(top_10_neighborhoods.to_string())
    print(f"\nMost expensive: {top_10_neighborhoods.index[0]} at ${top_10_neighborhoods['Average_Price'].iloc[0]:.2f}/night")
else:
    print("Required columns not found")

Q1: What are the top 10 most expensive neighborhoods by average price?

Top 10 Most Expensive Neighborhoods:
                         Average_Price  Listing_Count
neighbourhood                                        
Neighborhood highlights     474.489883           3509

Most expensive: Neighborhood highlights at $474.49/night


In [20]:
# Question 2: Average availability and price by room type
print("Q2: What's the average availability and price by room type?")
print("=" * 80)
if 'room_type' in df_clean.columns and 'price' in df_clean.columns and 'availability_365' in df_clean.columns:
    room_type_stats = df_clean.groupby('room_type').agg({
        'price': ['mean', 'median'],
        'availability_365': ['mean', 'median'],
        'id': 'count'
    }).round(2)
    room_type_stats.columns = ['Avg_Price', 'Median_Price', 'Avg_Availability', 'Median_Availability', 'Count']
    print("\nAverage Availability and Price by Room Type:")
    print(room_type_stats.to_string())
else:
    print("Required columns not found")

Q2: What's the average availability and price by room type?

Average Availability and Price by Room Type:
                 Avg_Price  Median_Price  Avg_Availability  Median_Availability  Count
room_type                                                                             
Entire home/apt     306.95         212.0            210.62                232.5   3736
Hotel room         9801.14         235.5            325.20                346.5     76
Private room        144.92          95.0            249.15                270.0   1919
Shared room         422.04          53.0            287.78                328.0     27


In [21]:
# Question 3: Host with the most listings
print("Q3: Which host has the most listings?")
print("=" * 80)
if 'host_id' in df_clean.columns and 'host_name' in df_clean.columns:
    host_listings = df_clean.groupby(['host_id', 'host_name']).size().reset_index(name='listing_count')
    host_listings = host_listings.sort_values('listing_count', ascending=False)
    top_host = host_listings.iloc[0]
    print(f"\nHost with most listings:")
    print(f"  Host Name: {top_host['host_name']}")
    print(f"  Host ID: {top_host['host_id']}")
    print(f"  Number of Listings: {top_host['listing_count']}")
    print("\nTop 10 Hosts by Number of Listings:")
    print(host_listings.head(10).to_string(index=False))
else:
    print("Required columns not found")

Q3: Which host has the most listings?

Host with most listings:
  Host Name: Ronald Alan
  Host ID: 542041520
  Number of Listings: 167

Top 10 Hosts by Number of Listings:
  host_id                           host_name  listing_count
542041520                         Ronald Alan            167
107434423                          Blueground            162
442029804                          Live Suite             79
446820235                   LuxurybookingsFZE             49
351395739                                Tony             47
267895362                               Rocky             46
482174715                          Suite Life             40
415840703                            Anyplace             38
 23719876                            Mingjing             37
410094341 Park Royal Powell Place At Nob Hill             36


In [22]:
# Question 4: Average price across different boroughs/districts
print("Q4: How does average price vary across different boroughs or districts?")
print("=" * 80)
if 'neighbourhood_group' in df_clean.columns and 'price' in df_clean.columns:
    borough_stats = df_clean.groupby('neighbourhood_group').agg({
        'price': ['mean', 'median', 'min', 'max'],
        'id': 'count'
    }).round(2)
    borough_stats.columns = ['Avg_Price', 'Median_Price', 'Min_Price', 'Max_Price', 'Listing_Count']
    borough_stats = borough_stats.sort_values('Avg_Price', ascending=False)
    print("\nPrice Statistics by Borough/District:")
    print(borough_stats.to_string())

    # Calculate price variance
    price_range = borough_stats['Avg_Price'].max() - borough_stats['Avg_Price'].min()
    print(f"\nPrice Range across districts: ${price_range:.2f}")
    print(f"Highest avg price: {borough_stats.index[0]} (${borough_stats['Avg_Price'].iloc[0]:.2f})")
    print(f"Lowest avg price: {borough_stats.index[-1]} (${borough_stats['Avg_Price'].iloc[-1]:.2f})")
else:
    print("Required columns not found")

Q4: How does average price vary across different boroughs or districts?
Required columns not found


In [23]:
# Question 5: Listings that have never been reviewed
print("Q5: How many listings have never been reviewed?")
print("=" * 80)
if 'number_of_reviews' in df_clean.columns:
    never_reviewed = df_clean[df_clean['number_of_reviews'] == 0]
    total_listings = len(df_clean)
    never_reviewed_count = len(never_reviewed)
    never_reviewed_pct = (never_reviewed_count / total_listings) * 100

    print(f"\nListings never reviewed: {never_reviewed_count:,} out of {total_listings:,}")
    print(f"Percentage: {never_reviewed_pct:.2f}%")

    # Additional insights about never-reviewed listings
    if 'price' in df_clean.columns:
        avg_price_never_reviewed = never_reviewed['price'].mean()
        avg_price_reviewed = df_clean[df_clean['number_of_reviews'] > 0]['price'].mean()
        print(f"\nAverage price of never-reviewed listings: ${avg_price_never_reviewed:.2f}")
        print(f"Average price of reviewed listings: ${avg_price_reviewed:.2f}")
        print(f"Price difference: ${abs(avg_price_never_reviewed - avg_price_reviewed):.2f}")
else:
    print("Required columns not found")

Q5: How many listings have never been reviewed?

Listings never reviewed: 1,150 out of 5,758
Percentage: 19.97%

Average price of never-reviewed listings: $707.56
Average price of reviewed listings: $296.75
Price difference: $410.81
